# FractionalIndex



This library helps with using fractional indexes to manage a collection of items, like a sqlite table or a directory of files.

What's a fractional index and why would you want it? Fractional indexes are _values_ designed to solve a particular problem: they let you manage an ordering over a set of items, via a key value for each item, in such a way that you can add a new item in between existing items without needing to update any item's key, and you can re-order a single item by updating only its key.

This is useful in situations where you need item keys to be stable because it expensive to update them or because other systems depend on them.

This library helps with two particular kinds of collections: rows in an sqlite table, or files in a directory.

The library works by providing an  **indexer**. The indexer treats the collection of items as an **external resource**. It reads the resource, but never modifies it. You the user are responsible for adding items to the collection. Whenever you add an item, you get the **key** to use for the item from the indexer.

## Example: sqlite, where a column holds a key

For instance, say you wanted to insert a new row in the table `mytable` in sqlite, but you wanted to give it a key so that it would appear in between two existing rows, which have values `a0` and `b0` in the column `order_key`.

You'd do it like so:

```python
from fractionalindex import SqliteIndexer

# init an indexer
sqliteindexer = SqliteIndexer(conn,'mytable', 'order_key')
# generate a new key between existing, adjoining keys
new_key = sqliteindexer.insert('a0','b0')
# insert the row
conn.execute(f'INSERT INTO mytable (order_key) VALUES (?)', (new_key,))
```

## Example: sqlite, where a column holds a key-derived name

The indexer's `insert()` method returns the key which is used for fractional indexing.

However, in your sqlite database, you might not want to use raw key values (like "a0") but instead prefer to use a **name** derived from a key, like "msg-a0".

To do this, pass the **name-to-key** to the indexer, and construct the name yourself when adding an item to the db:

```python
def name_to_key(name): 
    return name.split('-')[1] if (name is not None and '-' in name) else None
# init an indexer
sqliteindexer = SqliteIndexer(conn,'mytable', 'order_key',name_to_key)
# generate a new key between existing, adjoining keys
new_key = sqliteindexer.insert('msg-a0','msg-b0')
new_name = f"msg-{new_key}"
# insert the row
conn.execute(f'INSERT INTO mytable (order_key) VALUES (?)', (new_name,))
```

For sqlite, the library requires that the name ordering matches the key ordering, which will be the case with a constant prefix. But this is not required for the next example:


## Example: files, with names derived from keys

You can also use the library to define an ordering over a directory of files, like migration scripts.

Let's say you want script to have names like  `foo-a0.txt`, where `a0` is the key part. Then you'd define this name-to-key mapping function:

```python
def get_filekey(name):
    return name.sep(".")[0].sep("-")[1]
```

With this function defined, you could create a `FileIndexer` over the files in the current directory like so:

```python
from fractionalindex import FileIndexer

nfi = NamedFileIndexer(dir='.', name_to_key=get_filekey) # init the indexer
```

And you'd add a new file between the existing files "myfile-a0.txt" and "update-b0.txt", like so:

```python
newfile_key = nfi.insert(after="myfile-a0.txt",
                         before="update-b0.txt") # generate the new key
newfile_name = f"myfile-{newfile_key}"           # create the file name
Path(newfile_name).touch()                       # add a file to the directory
```

As with the database example, you are responsible creating the file. The indexer just provides the key to use in naming it.

## Extending to other resources

If you want to extend this library to create a new kind of indexer, the working model to keep in mind is fairly simple:

- the **Indexer** holds a reference to a **collection resource**, but does not modify it. 
- in the collection of items, every item has a **name**
- The indexer takes those names, and generates **keys**.
- internally, for each resource type, an **IndexAccessor** provides the first(), last(), before(), and after() accessors, which take names and return names


## fractional_indexing

This is a review of the underlying dependency library, used for generating fractional indexes

To review, the `fractional-indexing` pypi package provides primitives for generating and validating keys. The key-generating function takes two params, which are interpreted a bit like the components of a slice, in designating either zone between two existing indexes, or else defining the zone before or after an index

In [1]:
from fractional_indexing import generate_key_between
first = generate_key_between(None,None)

In [2]:
first

'a0'

In [3]:
second = generate_key_between(first,None)
second

'a1'

In [8]:
third = generate_key_between(second,None)
third

'a2'

The keys are strings and their lexicographic sort order is their index order:

In [7]:
[first,second,third] == sorted([first,second,third])

True

**Q: What about ambiguous specifications?** Like asking for a key between second-third vs simply after second?

In [11]:
generate_key_between(second,third)

'a1V'

In [12]:
generate_key_between(second,None)

'a2'

**Note regarding state**: The library is not managing a stateful collection of the keys which have been created. It only defining how one key values depend on each other. So generating repeatedly will always produce the same value.

In [9]:
[generate_key_between(second,None) for _ in range(10)][0]  == generate_key_between(second,None)

True

In [10]:
[first,second,third]

['a0', 'a1', 'a2']